In [2]:
!pip install gdown

In [3]:
import gdown

In [5]:
url = 'https://drive.google.com/uc?id=1PpjMOsbXxagL809U46RaTPU_Kc2alE3k'
output = 'BrainTumour.tar'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1PpjMOsbXxagL809U46RaTPU_Kc2alE3k
From (redirected): https://drive.google.com/uc?id=1PpjMOsbXxagL809U46RaTPU_Kc2alE3k&confirm=t&uuid=6161aeb4-cd8b-4aab-9a9f-560131cba791
To: /kaggle/working/BrainTumour.tar
100%|██████████| 7.61G/7.61G [00:46<00:00, 164MB/s] 


'BrainTumour.tar'

In [6]:
import tarfile

def extract_tar_file(tar_file_path, extract_path='.'):
    try:
        with tarfile.open(tar_file_path, 'r') as tar:
            tar.extractall(path=extract_path)
            print(f"Extracted all contents of {tar_file_path} to {extract_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

tar_file_path = '/kaggle/working/BrainTumour.tar'
extract_path = '/kaggle/working/'
extract_tar_file(tar_file_path, extract_path)


Extracted all contents of /kaggle/working/BrainTumour.tar to /kaggle/working/


In [7]:
import os

# Specify the file path
file_path = 'BrainTumour.tar'

# Check if the file exists before attempting to delete it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted.")
else:
    print(f"{file_path} does not exist.")

BrainTumour.tar has been deleted.


In [8]:
import nibabel as nib
from tqdm import tqdm
import matplotlib.pyplot as plt
import tarfile
import numpy as np
import os
from glob import glob

In [9]:
!pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.2 MB/s eta 0:00:0000:0100:01


In [10]:
from monai.utils import set_determinism

2024-09-04 13:46:12.743182: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 13:46:12.743282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 13:46:12.864360: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
os.mkdir('/kaggle/working/TrainVolumes')
os.mkdir('/kaggle/working/TrainSegmentation')
os.mkdir('/kaggle/working/TestVolumes')
os.mkdir('/kaggle/working/TestSegmentation')

In [12]:
import shutil

for i in range(1, 10):
  shutil.copy(f'/kaggle/working/Task01_BrainTumour/imagesTr/BRATS_00{i}.nii.gz', '/kaggle/working/TrainVolumes')

for i in range(1, 10):
  shutil.copy(f'/kaggle/working/Task01_BrainTumour/labelsTr/BRATS_00{i}.nii.gz', '/kaggle/working/TrainSegmentation')

for i in range(10, 100):
  shutil.copy(f'/kaggle/working/Task01_BrainTumour/imagesTr/BRATS_0{i}.nii.gz', '/kaggle/working//TrainVolumes')

for i in range(10, 100):
  shutil.copy(f'/kaggle/working/Task01_BrainTumour/labelsTr/BRATS_0{i}.nii.gz', '/kaggle/working/TrainSegmentation')

for i in range(100, 401):
  shutil.copy(f'/kaggle/working/Task01_BrainTumour/imagesTr/BRATS_{i}.nii.gz', '/kaggle/working//TrainVolumes')

for i in range(100, 401):
  shutil.copy(f'/kaggle/working/Task01_BrainTumour/labelsTr/BRATS_{i}.nii.gz', '/kaggle/working/TrainSegmentation')



In [13]:
for i in range(401, 451):
  shutil.copy(f'/kaggle/working/Task01_BrainTumour/imagesTr/BRATS_{i}.nii.gz', '/kaggle/working/TestVolumes')

for i in range(401, 451):
  shutil.copy(f'/kaggle/working/Task01_BrainTumour/labelsTr/BRATS_{i}.nii.gz', '/kaggle/working/TestSegmentation')


In [14]:
from monai.transforms import Compose, LoadImaged, Resized, ToTensord, Spacingd, Orientationd, ScaleIntensityRanged, CropForegroundd, EnsureChannelFirstd
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism, first
import matplotlib.pyplot as plt

In [15]:
def prepare(in_dir, pixdim = (1.5, 1.5, 1.0),  spatial_size =[128, 128, 128], cache  = False):
  """
  Preprocessing Function
  """

  set_determinism(seed=0)

  path_train_volumes = sorted(glob(os.path.join(in_dir, 'TrainVolumes', "*.nii.gz")))
  path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))

  path_test_volume = sorted(glob(os.path.join(in_dir, 'TestVolumes', "*.nii.gz")))
  path_test_segmentation = sorted(glob(os.path.join(in_dir, "TestSegmentation", "*.nii.gz")))

  train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in
                 zip(path_train_volumes, path_train_segmentation)]
  test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in
                zip(path_test_volume, path_test_segmentation)]

  train_transforms = Compose(
      [
          LoadImaged(keys=["vol", "seg"]),
          EnsureChannelFirstd(keys=["vol", "seg"]),
          Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
          Orientationd(keys=["vol", "seg"], axcodes="RAS"),
          CropForegroundd(keys=["vol", "seg"], source_key="vol"),
          CropForegroundd(keys=["vol", "seg"], source_key="seg"),
          Resized(keys=["vol", "seg"], spatial_size=spatial_size),
          ToTensord(keys=["vol", "seg"])

      ]
  )

  test_transforms = Compose([
      LoadImaged(keys=["vol", "seg"]),
      EnsureChannelFirstd(keys=["vol", "seg"]),
      Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
      Orientationd(keys=["vol", "seg"], axcodes="RAS"),
      CropForegroundd(keys=["vol", "seg"], source_key="vol"),
      Resized(keys=["vol", "seg"], spatial_size=spatial_size),
      ToTensord(keys=["vol", "seg"])
  ])

  if cache:
        train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = CacheDataset(data=test_files, transform=test_transforms, cache_rate=1.0)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader

  else:
        train_ds = Dataset(data=train_files, transform=train_transforms)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = Dataset(data=test_files, transform=test_transforms)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader




In [16]:
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
import numpy as np
from monai.losses import DiceLoss
from tqdm import tqdm


In [17]:

def dice_metric(predicted, target):
    '''
    In this function we take `predicted` and `target` (label) to calculate the dice coeficient then we use it
    to calculate a metric value for the training and the validation.
    '''
    dice_value = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
    value = 1 - dice_value(predicted, target).item()
    return value

def hausdorff_metric(predicted, target):
    """
    This function calculates the Hausdorff distance between `predicted` and `target` (label).
    It returns a metric value for training and validation.
    """
    # Initialize Hausdorff Distance Metric with 95th percentile
    hd_metric = HausdorffDistanceMetric(include_background=True, percentile=95)
    
    # Compute the Hausdorff distance
    hausdorff_distance = hd_metric(predicted, target)
    
    # Return the computed distance
    return hausdorff_distance.item()


def calculate_weights(val1, val2):
    '''
    In this function we take the number of the background and the forgroud pixels to return the `weights`
    for the cross entropy loss values.
    '''
    count = np.array([val1, val2])
    summ = count.sum()
    weights = count/summ
    weights = 1/weights
    summ = weights.sum()
    weights = weights/summ
    return torch.tensor(weights, dtype=torch.float32)

def train(model, data_in, loss, optim, max_epochs, model_dir, test_interval=1 , device=torch.device("cuda:0")):
    best_metric = -1
    best_metric_epoch = -1
    save_loss_train = []
    save_loss_test = []
    save_metric_train = []
    save_metric_test = []
    train_loader, test_loader = data_in

    for epoch in range(max_epochs):
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        model.train()
        train_epoch_loss = 0
        train_step = 0
        epoch_metric_train = 0
        for batch_data in train_loader:

            train_step += 1

            volume = batch_data["vol"]
            label = batch_data["seg"]
            label = label != 0
            volume, label = (volume.to(device), label.to(device))

            optim.zero_grad()
            outputs = model(volume)

            train_loss = loss(outputs, label)

            train_loss.backward()
            optim.step()

            train_epoch_loss += train_loss.item()
            print(
                f"{train_step}/{len(train_loader) // train_loader.batch_size}, "
                f"Train_loss: {train_loss.item():.4f}")

            train_metric = dice_metric(outputs, label)
            epoch_metric_train += train_metric
            print(f'Train_dice: {train_metric:.4f}')

        print('-'*20)

        train_epoch_loss /= train_step
        print(f'Epoch_loss: {train_epoch_loss:.4f}')
        save_loss_train.append(train_epoch_loss)
        np.save(os.path.join(model_dir, 'loss_train.npy'), save_loss_train)

        epoch_metric_train /= train_step
        print(f'Epoch_metric: {epoch_metric_train:.4f}')

        save_metric_train.append(epoch_metric_train)
        np.save(os.path.join(model_dir, 'metric_train.npy'), save_metric_train)

        if (epoch + 1) % test_interval == 0:

            model.eval()
            with torch.no_grad():
                test_epoch_loss = 0
                test_metric = 0
                epoch_metric_test = 0
                test_step = 0

                for test_data in test_loader:

                    test_step += 1

                    test_volume = test_data["vol"]
                    test_label = test_data["seg"]
                    test_label = test_label != 0
                    test_volume, test_label = (test_volume.to(device), test_label.to(device),)

                    test_outputs = model(test_volume)

                    test_loss = loss(test_outputs, test_label)
                    test_epoch_loss += test_loss.item()
                    test_metric = dice_metric(test_outputs, test_label)
                    epoch_metric_test += test_metric


                test_epoch_loss /= test_step
                print(f'test_loss_epoch: {test_epoch_loss:.4f}')
                save_loss_test.append(test_epoch_loss)
                np.save(os.path.join(model_dir, 'loss_test.npy'), save_loss_test)

                epoch_metric_test /= test_step
                print(f'test_dice_epoch: {epoch_metric_test:.4f}')
                save_metric_test.append(epoch_metric_test)
                np.save(os.path.join(model_dir, 'metric_test.npy'), save_metric_test)

                if epoch_metric_test > best_metric:
                    best_metric = epoch_metric_test
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), os.path.join(
                        model_dir, "best_metric_model.pth"))

                print(
                    f"current epoch: {epoch + 1} current mean dice: {test_metric:.4f}"
                    f"\nbest mean dice: {best_metric:.4f} "
                    f"at epoch: {best_metric_epoch}"
                )


    print(
        f"train completed, best_metric: {best_metric:.4f} "
        f"at epoch: {best_metric_epoch}")


def show_patient(data, SLICE_NUMBER=1, train=True, test=False):
    """
    This function is to show one patient from your datasets, so that you can si if the it is okay or you need
    to change/delete something.

    `data`: this parameter should take the patients from the data loader, which means you need to can the function
    prepare first and apply the transforms that you want after that pass it to this function so that you visualize
    the patient with the transforms that you want.
    `SLICE_NUMBER`: this parameter will take the slice number that you want to display/show
    `train`: this parameter is to say that you want to display a patient from the training data (by default it is true)
    `test`: this parameter is to say that you want to display a patient from the testing patients.
    """

    check_patient_train, check_patient_test = data

    view_train_patient = first(check_patient_train)
    view_test_patient = first(check_patient_test)


    if train:
        plt.figure("Visualization Train", (12, 6))
        plt.subplot(1, 2, 1)
        plt.title(f"vol {SLICE_NUMBER}")
        plt.imshow(view_train_patient["vol"][0, 0, :, :, SLICE_NUMBER], cmap="gray")

        plt.subplot(1, 2, 2)
        plt.title(f"seg {SLICE_NUMBER}")
        plt.imshow(view_train_patient["seg"][0, 0, :, :, SLICE_NUMBER])
        plt.show()

    if test:
        plt.figure("Visualization Test", (12, 6))
        plt.subplot(1, 2, 1)
        plt.title(f"vol {SLICE_NUMBER}")
        plt.imshow(view_test_patient["vol"][0, 0, :, :, SLICE_NUMBER], cmap="gray")

        plt.subplot(1, 2, 2)
        plt.title(f"seg {SLICE_NUMBER}")
        plt.imshow(view_test_patient["seg"][0, 0, :, :, SLICE_NUMBER])
        plt.show()


def calculate_pixels(data):
    val = np.zeros((1, 2))

    for batch in tqdm(data):
        batch_label = batch["seg"] != 0
        _, count = np.unique(batch_label, return_counts=True)

        if len(count) == 1:
            count = np.append(count, 0)
        val += count

    print('The last values:', val)
    return val

In [18]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss
from monai.metrics import HausdorffDistanceMetric

In [19]:
model_dir = '/kaggle/working/results/results'

In [20]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss
import torch

In [ ]:


data_dir = '/kaggle/working/'
model_dir = '/kaggle/working/results/results'
data_in = prepare(data_dir, cache=True)

device = torch.device("cuda:0")

model = model = UNet(
    spatial_dims=3,
    in_channels=4,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

#loss_function = DiceCELoss(to_onehot_y=True, sigmoid=True, squared_pred=True, ce_weight=calculate_weights(1792651250,2510860).to(device))
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)


train(model, data_in, loss_function, optimizer, 600, model_dir)

/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
Loading dataset: 100%|██████████| 50/50 [02:12<00:00,  2.65s/it]


----------
epoch 1/600
1/400, Train_loss: 0.3515
Train_dice: 0.6485
2/400, Train_loss: 0.3371
Train_dice: 0.6629
3/400, Train_loss: 0.3546
Train_dice: 0.6454
4/400, Train_loss: 0.3329
Train_dice: 0.6671
5/400, Train_loss: 0.3802
Train_dice: 0.6198
6/400, Train_loss: 0.3435
Train_dice: 0.6565
7/400, Train_loss: 0.3961
Train_dice: 0.6039
8/400, Train_loss: 0.3442
Train_dice: 0.6558
9/400, Train_loss: 0.3361
Train_dice: 0.6639
10/400, Train_loss: 0.3689
Train_dice: 0.6311
11/400, Train_loss: 0.3349
Train_dice: 0.6651
12/400, Train_loss: 0.3357
Train_dice: 0.6643
13/400, Train_loss: 0.3340
Train_dice: 0.6660
14/400, Train_loss: 0.3336
Train_dice: 0.6664
15/400, Train_loss: 0.3423
Train_dice: 0.6577
16/400, Train_loss: 0.3411
Train_dice: 0.6589
17/400, Train_loss: 0.3524
Train_dice: 0.6476
18/400, Train_loss: 0.3308
Train_dice: 0.6692
19/400, Train_loss: 0.4028
Train_dice: 0.5972
20/400, Train_loss: 0.3395
Train_dice: 0.6605
21/400, Train_loss: 0.3332
Train_dice: 0.6668
22/400, Train_loss: 

In [ ]:
#loss_function = DiceCELoss(to_onehot_y=True, sigmoid=True, squared_pred=True, ce_weight=calculate_weights(1792651250,2510860).to(device))
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)

train(model, data_in, loss_function, optimizer, 600, model_dir)

In [ ]:
model = UNet().to(device)

#loss_function = DiceCELoss(to_onehot_y=True, sigmoid=True, squared_pred=True, ce_weight=calculate_weights(1792651250,2510860).to(device))
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)


train(model, data_in, loss_function, optimizer, 600, model_dir)

In [ ]:
import torch
import torch.nn as nn

class BifurcatingLayer(nn.Module):
    def __init__(self, channels):
        super(BifurcatingLayer, self).__init__()
        self.main_path = nn.Conv3d(channels, channels, kernel_size=1)
        self.bifurcation1 = nn.Conv3d(channels, channels, kernel_size=1)
        self.bifurcation2 = nn.Conv3d(channels, channels, kernel_size=1)

    def forward(self, x):
        main = self.main_path(x)
        b1 = self.bifurcation1(x)
        b2 = self.bifurcation2(x)
        return torch.where(torch.abs(main) > 1, b1, b2)

class ADN(nn.Module):
    def __init__(self, channels, dropout_prob=0.0):
        super(ADN, self).__init__()
        self.N = nn.BatchNorm3d(channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.D = nn.Dropout(p=dropout_prob, inplace=False)
        self.A = BifurcatingLayer(channels)

    def forward(self, x):
        x = self.N(x)
        x = self.D(x)
        x = self.A(x)
        return x

# The rest of the code remains the same

class Convolution(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, is_transposed=False, use_adn=True):
        super(Convolution, self).__init__()
        if is_transposed:
            self.conv = nn.ConvTranspose3d(in_channels, out_channels, kernel_size, stride, padding, output_padding=stride//2)
        else:
            self.conv = nn.Conv3d(in_channels, out_channels, kernel_size, stride, padding)
        self.use_adn = use_adn
        if use_adn:
            self.adn = ADN(out_channels)

    def forward(self, x):
        x = self.conv(x)
        if self.use_adn:
            x = self.adn(x)
        return x

class ResidualUnit(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualUnit, self).__init__()
        self.conv = nn.Sequential(
            Convolution(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            Convolution(out_channels, out_channels, kernel_size=3, stride=1, padding=1, use_adn=False)
        )
        if stride != 1 or in_channels != out_channels:
            self.residual = Convolution(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, use_adn=False)
        else:
            self.residual = nn.Identity()

    def forward(self, x):
        return self.conv(x) + self.residual(x)

class SkipConnection(nn.Module):
    def __init__(self, submodule):
        super(SkipConnection, self).__init__()
        self.submodule = submodule

    def forward(self, x):
        return torch.cat([x, self.submodule(x)], dim=1)

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.model = nn.Sequential(
            ResidualUnit(4, 16, stride=2),
            SkipConnection(
                nn.Sequential(
                    ResidualUnit(16, 32, stride=2),
                    SkipConnection(
                        nn.Sequential(
                            ResidualUnit(32, 64, stride=2),
                            SkipConnection(
                                nn.Sequential(
                                    ResidualUnit(64, 128, stride=2),
                                    SkipConnection(
                                        ResidualUnit(128, 256, stride=1)
                                    ),
                                    nn.Sequential(
                                        Convolution(384, 64, kernel_size=3, stride=2, padding=1, is_transposed=True),
                                        ResidualUnit(64, 64)
                                    )
                                )
                            ),
                            nn.Sequential(
                                Convolution(128, 32, kernel_size=3, stride=2, padding=1, is_transposed=True),
                                ResidualUnit(32, 32)
                            )
                        )
                    ),
                    nn.Sequential(
                        Convolution(64, 16, kernel_size=3, stride=2, padding=1, is_transposed=True),
                        ResidualUnit(16, 16)
                    )
                )
            ),
            nn.Sequential(
                Convolution(32, 2, kernel_size=3, stride=2, padding=1, is_transposed=True),
                ResidualUnit(2, 2)
            )
        )

    def forward(self, x):
        return self.model(x)

# Create the model
model = UNet()

# Print the model structure
print(model)

# You can now use this model for training or inference
# For example:
input_tensor = torch.randn(1, 4, 64, 64, 64)  # Batch size 1, 4 input channels, 64x64x64 volume
output = model(input_tensor)
print(output.shape)

In [ ]:
 if cache:
    train_ds = CacheDataset(data = train_files, transform = train_transforms, cache_rate = 1.0)
    train_loader = DataLoader(train_ds, batch_size = 1)

    test_ds = CacheDataset(data = test_files, transform = test_transform, cache_rate = 1.0)
    test_loader = DataLoader(test_ds, batch_size = 1)

    return train_loader, test_loader

In [21]:
os.mkdir('/kaggle/working/results')
os.mkdir('/kaggle/working/results/results')

In [ ]:
from monai.inferers import sliding_window_inference

In [ ]:
in_dir = '/kaggle/working/'
model_dir = '/kaggle/working/results/results'

In [ ]:
train_loss = np.load(os.path.join(model_dir, 'loss_train.npy'))
train_metric = np.load(os.path.join(model_dir, 'metric_train.npy'))
test_loss = np.load(os.path.join(model_dir, 'loss_test.npy'))
test_metric = np.load(os.path.join(model_dir, 'metric_test.npy'))

In [ ]:
plt.figure("Results 14th Aufust Brain Tumour with unet", (12, 6))
plt.subplot(2, 2, 1)
plt.title("Train dice loss")
x = [i + 1 for i in range(len(train_loss))]
y = train_loss
plt.xlabel("epoch")
plt.plot(x, y)

plt.subplot(2, 2, 2)
plt.title("Train metric DICE")
x = [i + 1 for i in range(len(train_metric))]
y = train_metric
plt.xlabel("epoch")
plt.plot(x, y)

plt.subplot(2, 2, 3)
plt.title("Test dice loss")
x = [i + 1 for i in range(len(test_loss))]
y = test_loss
plt.xlabel("epoch")
plt.plot(x, y)

plt.subplot(2, 2, 4)
plt.title("Test metric DICE")
x = [i + 1 for i in range(len(test_metric))]
y = test_metric
plt.xlabel("epoch")
plt.plot(x, y)

plt.show()

In [ ]:
path_train_volumes = sorted(glob(os.path.join(in_dir, "TrainVolumes", "*.nii.gz")))
path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSegmentation", "*.nii.gz")))

path_test_volumes = sorted(glob(os.path.join(in_dir, "TestVolumes", "*.nii.gz")))
path_test_segmentation = sorted(glob(os.path.join(in_dir, "TestSegmentation", "*.nii.gz")))

train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]
test_files = test_files[0:9]

In [ ]:
  test_transforms = Compose([
      LoadImaged(keys=["vol", "seg"]),
      EnsureChannelFirstd(keys=["vol", "seg"]),
      Spacingd(keys=["vol", "seg"], pixdim=(1.5,1.5,1.0), mode=("bilinear", "nearest")),
      Orientationd(keys=["vol", "seg"], axcodes="RAS"),
      CropForegroundd(keys=["vol", "seg"], source_key="vol"),
      Resized(keys=["vol", "seg"], spatial_size=[128,128,128]),   
      ToTensord(keys=["vol", "seg"])
  ])

In [ ]:
test_ds = Dataset(data=test_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1)

In [ ]:
device = torch.device("cuda:0")
model = UNet(
    spatial_dims=3,
    in_channels=4,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

In [ ]:
model.load_state_dict(torch.load(
    os.path.join(model_dir, "best_metric_model.pth")))
model.eval()

In [ ]:
sw_batch_size = 4
roi_size = (128, 128, 64)
with torch.no_grad():
    test_patient = first(test_loader)
    t_volume = test_patient['vol']
    #t_segmentation = test_patient['seg']
    
    test_outputs = sliding_window_inference(t_volume.to(device), roi_size, sw_batch_size, model)
    sigmoid_activation = Activations(sigmoid=True)
    test_outputs = sigmoid_activation(test_outputs)
    test_outputs = test_outputs > 0.53
        
    for i in range(80, 90):
        # plot the slice [:, :, 80]
        plt.figure("check", (18, 6))
        plt.subplot(1, 3, 1)
        plt.title(f"image {i}")
        plt.imshow(test_patient["vol"][0, 0, :, :, i], cmap="gray")
        plt.subplot(1, 3, 2)
        plt.title(f"label {i}")
        plt.imshow(test_patient["seg"][0, 0, :, :, i] != 0)
        plt.subplot(1, 3, 3)
        plt.title(f"output {i}")
        plt.imshow(test_outputs.detach().cpu()[0, 1, :, :, i])
        plt.show()


In [ ]:
from monai.transforms import Activations

In [ ]:
import torch
import torch.nn as nn

class PitchforkActivation(nn.Module):
    def __init__(self, bifurcation_point=0):
        super(PitchforkActivation, self).__init__()
        self.bifurcation_point = bifurcation_point

    def forward(self, x):
        return torch.where(x < self.bifurcation_point, 
                           -torch.pow(x - self.bifurcation_point, 3),
                           torch.pow(x - self.bifurcation_point, 3))

class BifurcatingLayer(nn.Module):
    def __init__(self, channels):
        super(BifurcatingLayer, self).__init__()
        self.main_path = nn.Conv3d(channels, channels, kernel_size=1)
        self.bifurcation1 = nn.Conv3d(channels, channels, kernel_size=1)
        self.bifurcation2 = nn.Conv3d(channels, channels, kernel_size=1)

    def forward(self, x):
        main = self.main_path(x)
        b1 = self.bifurcation1(x)
        b2 = self.bifurcation2(x)
        return torch.where(torch.abs(main) > 1, b1, b2)

class ADN(nn.Module):
    def __init__(self, channels, dropout_prob=0.0):
        super(ADN, self).__init__()
        self.N = nn.BatchNorm3d(channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.D = nn.Dropout(p=dropout_prob, inplace=False)
        self.A = PitchforkActivation()
        self.B = BifurcatingLayer(channels)

    def forward(self, x):
        x = self.N(x)
        x = self.D(x)
        x = self.A(x)
        x = self.B(x)
        return x

class Convolution(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, is_transposed=False, use_adn=True):
        super(Convolution, self).__init__()
        if is_transposed:
            self.conv = nn.ConvTranspose3d(in_channels, out_channels, kernel_size, stride, padding, output_padding=stride//2)
        else:
            self.conv = nn.Conv3d(in_channels, out_channels, kernel_size, stride, padding)
        self.use_adn = use_adn
        if use_adn:
            self.adn = ADN(out_channels)

    def forward(self, x):
        x = self.conv(x)
        if self.use_adn:
            x = self.adn(x)
        return x

class ResidualUnit(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualUnit, self).__init__()
        self.conv = nn.Sequential(
            Convolution(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            Convolution(out_channels, out_channels, kernel_size=3, stride=1, padding=1, use_adn=False)
        )
        if stride != 1 or in_channels != out_channels:
            self.residual = Convolution(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, use_adn=False)
        else:
            self.residual = nn.Identity()

    def forward(self, x):
        return self.conv(x) + self.residual(x)

class SkipConnection(nn.Module):
    def __init__(self, submodule):
        super(SkipConnection, self).__init__()
        self.submodule = submodule

    def forward(self, x):
        return torch.cat([x, self.submodule(x)], dim=1)

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.model = nn.Sequential(
            ResidualUnit(4, 16, stride=2),
            SkipConnection(
                nn.Sequential(
                    ResidualUnit(16, 32, stride=2),
                    SkipConnection(
                        nn.Sequential(
                            ResidualUnit(32, 64, stride=2),
                            SkipConnection(
                                nn.Sequential(
                                    ResidualUnit(64, 128, stride=2),
                                    SkipConnection(
                                        ResidualUnit(128, 256, stride=1)
                                    ),
                                    nn.Sequential(
                                        Convolution(384, 64, kernel_size=3, stride=2, padding=1, is_transposed=True),
                                        ResidualUnit(64, 64)
                                    )
                                )
                            ),
                            nn.Sequential(
                                Convolution(128, 32, kernel_size=3, stride=2, padding=1, is_transposed=True),
                                ResidualUnit(32, 32)
                            )
                        )
                    ),
                    nn.Sequential(
                        Convolution(64, 16, kernel_size=3, stride=2, padding=1, is_transposed=True),
                        ResidualUnit(16, 16)
                    )
                )
            ),
            nn.Sequential(
                Convolution(32, 2, kernel_size=3, stride=2, padding=1, is_transposed=True),
                ResidualUnit(2, 2)
            )
        )

    def forward(self, x):
        return self.model(x)

# Create the model
model = UNet()

# Print the model structure
print(model)

# Adaptive Learning Rate Scheduler
class PitchforkLRScheduler(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, bifurcation_point, last_epoch=-1):
        self.bifurcation_point = bifurcation_point
        super(PitchforkLRScheduler, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.bifurcation_point:
            return [base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr * (1 + abs(self.last_epoch - self.bifurcation_point))
                    for base_lr in self.base_lrs]

# Example usage:
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = PitchforkLRScheduler(optimizer, bifurcation_point=50)

# You can now use this model for training or inference
# For example:
# input_tensor = torch.randn(1, 4, 64, 64, 64)  # Batch size 1, 4 input channels, 64x64x64 volume
# output = model(input_tensor)
# print(output.shape)